In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report 

In [0]:
df = pd.read_csv('Beans_Total.csv')

In [5]:
df.head(5)

,No,fileName,fileName_seeds,Type,Hu_1,Hu_2,Hu_3,Hu_4,Hu_5,Hu_6,Hu_7,HRL_1,HRL_2,HRL_3,HRL_4,HRL_5,HRL_6,HRL_7,HRL_8,HRL_9,HRL_10,HRL_11,HRL_12,HRL_13
0,1,DSC09725.JPG,66,1,0.002957,2.140000e-07,1.780000e-10,3.520000e-11,-2.570000e-21,1.620000e-14,-1.070000e-21,0.190214,65.264548,0.967536,1005.238808,0.627581,67.028161,3955.690686,4.887311,6.423181,0.002829,2.364985,-0.506230,0.993370
1,2,DSC09725.JPG,69,1,0.002068,3.470000e-07,2.260000e-10,6.140000e-11,6.890000e-21,3.610000e-14,-2.240000e-21,0.135258,68.647570,0.976882,1485.077360,0.593441,97.872322,5871.661870,5.070097,6.698752,0.002232,2.451770,-0.468187,0.991361
2,3,DSC09725.JPG,70,1,0.002632,6.810000e-07,1.320000e-10,2.650000e-11,-1.050000e-21,1.010000e-14,-1.160000e-21,0.136973,55.419592,0.973069,1029.134878,0.608767,81.162359,4061.119919,4.887796,6.438389,0.002737,2.351940,-0.469432,0.989995
3,4,DSC09725.JPG,71,1,0.002453,3.130000e-07,1.770000e-10,3.110000e-11,1.000000e-21,-6.960000e-15,2.080000e-21,0.170119,76.505316,0.972894,1411.343958,0.601557,83.833087,5568.870516,5.106670,6.769519,0.002326,2.534172,-0.483859,0.993133
4,5,DSC09725.JPG,72,1,0.002222,4.110000e-07,1.920000e-11,2.740000e-12,-2.070000e-24,2.700000e-16,1.980000e-23,0.148740,57.336303,0.979066,1369.665113,0.605767,91.741878,5421.324149,4.687771,6.258966,0.002659,2.385366,-0.444420,0.985478


In [6]:
df.Type.value_counts()

0    4533
1    1295
Name: Type, dtype: int64

# baseline

In [7]:
df_suffle = df.sample(frac=1)
X_b = df_suffle.iloc[:,4:].reset_index(drop = True)
y_b = df_suffle.iloc[:,3].reset_index(drop = True)

X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(X_b, y_b, test_size=0.3, stratify = y_b, random_state=2019)

clf_b = LogisticRegression(solver='liblinear', random_state = 2019)
score_b = cross_val_score(clf_b, X_train_b, y_train_b, cv = 10, scoring='f1')
print(f'Base validation f1 score: {score_b.mean():.2f}')

clf_b.fit(X_train_b, y_train_b)
train_pred_b = clf_b.predict(X_train_b)
test_pred_b = clf_b.predict(X_test_b)
print(f'Base f1 training score: {f1_score(y_train_b, train_pred_b):.2f}')
print(f'Base f1 testing score: {f1_score(y_test_b, test_pred_b):.2f}')
pd.crosstab(y_test_b, clf_b.predict(X_test_b), rownames=['Actual'], colnames=['Predicted'])

Base validation f1 score: 0.65
Base f1 training score: 0.61
Base f1 testing score: 0.60


Predicted,0,1
Actual,,
0,1289,71
1,191,198


In [8]:
print(classification_report(y_test_b, clf_b.predict(X_test_b)))     

              precision    recall  f1-score   support

           0       0.87      0.95      0.91      1360
           1       0.74      0.51      0.60       389

    accuracy                           0.85      1749
   macro avg       0.80      0.73      0.75      1749
weighted avg       0.84      0.85      0.84      1749



# under sampling

In [0]:
zero_cls = df[df.Type == 0]
one_cls = df[df.Type == 1]

In [0]:
zero_cls = zero_cls.sample(frac=1).reset_index(drop = True)
zero_cls = zero_cls.iloc[:one_cls.shape[0],:]

In [0]:
under_sampling_df = pd.concat([one_cls, zero_cls]).reset_index(drop=True)
under_sampling_df = under_sampling_df.sample(frac=1)

In [12]:
under_sampling_df.Type.value_counts()

1    1295
0    1295
Name: Type, dtype: int64

In [0]:
X = under_sampling_df.iloc[:,4:].reset_index(drop = True)
y = under_sampling_df.iloc[:,3].reset_index(drop = True)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify = y, random_state=2019)

## Logistic regression classifier

In [16]:
clf = LogisticRegression(solver='liblinear', random_state = 2019)

score = cross_val_score(clf, X_train, y_train, cv = 10, scoring='f1')
print(f'LogisticRegression cross validation f1 score: {score.mean():.2f}')

clf.fit(X_train,y_train)
print(f'f1 training score: {f1_score(y_train, clf.predict(X_train)):.2f}')
print(f'f1 testing score: {f1_score(y_test, clf.predict(X_test)):.2f}')
# print(f'training score: {clf.score(X_train,y_train):.2f}')
# print(f'testing score: {clf.score(X_test,y_test):.2f}')
# confusion_matrix(clf.predict(X_test),y_test)
pd.crosstab(y_test, clf.predict(X_test), rownames=['Actual'], colnames=['Predicted'])

LogisticRegression cross validation f1 score: 0.87
f1 training score: 0.88
f1 testing score: 0.86


Predicted,0,1
Actual,,
0,315,74
1,39,349


In [17]:
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.89      0.81      0.85       389
           1       0.83      0.90      0.86       388

    accuracy                           0.85       777
   macro avg       0.86      0.85      0.85       777
weighted avg       0.86      0.85      0.85       777



# with features scaling

### cross validation

In [18]:
# with features scaling
pipe = make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear', random_state = 2019))
score = cross_val_score(pipe, X_train, y_train, cv = 10, scoring='f1')
print(f'LogisticRegression f1 cross validation score: {score.mean():.2f}')

LogisticRegression f1 cross validation score: 0.88


### training and test score

In [19]:
pipe.fit(X_train,y_train)
print(f'f1 training score: {f1_score(y_train,pipe.predict(X_train)):.2f}')
print(f'f1 testing score: {f1_score(y_test, pipe.predict(X_test)):.2f}')
# confusion_matrix(pipe.predict(X_test),y_test)
pd.crosstab(y_test, pipe.predict(X_test), rownames=['Actual'], colnames=['Predicted'])

f1 training score: 0.88
f1 testing score: 0.87


Predicted,0,1
Actual,,
0,329,60
1,42,346


In [20]:
print(classification_report(y_test, pipe.predict(X_test)))

              precision    recall  f1-score   support

           0       0.89      0.85      0.87       389
           1       0.85      0.89      0.87       388

    accuracy                           0.87       777
   macro avg       0.87      0.87      0.87       777
weighted avg       0.87      0.87      0.87       777



# Stack ensembles

In [21]:
# For parameters of RF and SVM use from (3) and (4)
voting = VotingClassifier([('rf',RandomForestClassifier(n_estimators = 100, random_state = 2019)),\
                          ('svm',SVC(probability=True, random_state = 2019)),\
                           ('lr',LogisticRegression(solver='liblinear', random_state = 2019))],\
                          voting='soft')

base_clf = make_pipeline(StandardScaler(),voting)
x_train_transform = cross_val_predict(base_clf, X_train, y_train, cv=10, method='transform')
meta_clf = LogisticRegression(solver='liblinear', random_state = 2019).fit(x_train_transform,y_train)

# transform X_test
base_clf_transformer = base_clf.fit(X_train, y_train)
X_test_transform = base_clf_transformer.transform(X_test)

# compare accuracy between train and test data
print(f'f1 training_accuracy: {f1_score(y_train, meta_clf.predict(x_train_transform)):.2f}')
print(f'f1 test_accuracy: {f1_score(y_test, meta_clf.predict(X_test_transform)):.2f}')
pd.crosstab(y_test, meta_clf.predict(X_test_transform), rownames=['Actual'], colnames=['Predicted'])

f1 training_accuracy: 0.90
f1 test_accuracy: 0.88


Predicted,0,1
Actual,,
0,347,42
1,49,339


In [25]:
x_train_transform.shape

(1813, 6)

In [27]:
x_train_transform[0]

array([0.69      , 0.31      , 0.7755738 , 0.2244262 , 0.68059184,
       0.31940816])

In [26]:
X_test_transform.shape

(777, 6)

In [22]:
print(classification_report(y_test, meta_clf.predict(X_test_transform)))

              precision    recall  f1-score   support

           0       0.88      0.89      0.88       389
           1       0.89      0.87      0.88       388

    accuracy                           0.88       777
   macro avg       0.88      0.88      0.88       777
weighted avg       0.88      0.88      0.88       777

